In [1]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
import pandas as pd
import re

db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

In [2]:

TABLES_MAP = {
    'fi_cad_fi_hist_admin': ['cnpj_admin', 'admin', 'dt_ini_admin', 'dt_fim_admin'],
    'fi_cad_fi_hist_auditor': ['cnpj_auditor', 'auditor', 'dt_fim_auditor'],
    'fi_cad_fi_hist_classe': ['classe', 'dt_fim_classe'],
    'fi_cad_fi_hist_condom': ['condom', 'dt_fim_condom'],
    'fi_cad_fi_hist_controlador': ['cnpj_controlador', 'controlador', 'dt_fim_controlador'],
    'fi_cad_fi_hist_custodiante': ['cnpj_custodiante', 'custodiante', 'dt_fim_custodiante'],
    'fi_cad_fi_hist_denom_social': ['denom_social', 'dt_fim_denom_social'],
    'fi_cad_fi_hist_denom_comerc': ['denom_comerc', 'dt_fim_denom_comerc'],
    'fi_cad_fi_hist_diretor_resp': ['diretor', 'dt_fim_diretor'],
    'fi_cad_fi_hist_exclusivo': ['fundo_exclusivo', 'dt_fim_st_exclusivo'],
    'fi_cad_fi_hist_fic': ['fundo_cotas', 'dt_fim_st_cotas'],
    'fi_cad_fi_hist_gestor': ['pf_pj_gestor', 'cpf_cnpj_gestor', 'gestor', 'dt_ini_gestor', 'dt_fim_gestor'],
    'fi_cad_fi_hist_publico_alvo': ['publico_alvo', 'dt_fim_publico_alvo'],
    'fi_cad_fi_hist_rentab': ['rentab_fundo', 'dt_fim_rentab'],
    'fi_cad_fi_hist_sit': ['sit', 'dt_ini_sit', 'dt_fim_sit'],
    'fi_cad_fi_hist_taxa_adm': ['taxa_adm', 'inf_taxa_adm'],
    'fi_cad_fi_hist_taxa_perfm': ['vl_taxa_perfm', 'ds_taxa_perfm'],
    'fi_cad_fi_hist_trib_lprazo': ['trib_lprazo', 'dt_fim_st_trib_lprazo']
}

In [3]:
from copy import copy
from abc import ABC
# class Tables(ABC): pass
# _tables = Tables()
_tables = {}
for table_name in TABLES_MAP.keys():
    _tables[table_name] = read_sql(f"SELECT * FROM cvm.{table_name}")
    # setattr(_tables, table_name, read_sql(f"SELECT * FROM cvm.{table_name}"))

In [4]:
final_cols = [
    'cnpj_fundo', 'dt_reg', 'dt_ini', 
    'cnpj_admin', 'admin', 
    'cnpj_auditor', 'auditor',
    'cnpj_custodiante', 'custodiante',
    'classe', 'condom', 
    'cnpj_controlador', 'controlador',
    'denom_comerc', 'denom_social', 'diretor',
    'fundo_exclusivo', 'fundo_cotas',
    'pf_pj_gestor', 'cpf_cnpj_gestor', 'gestor',
    'publico_alvo', 'rentab_fundo', 'sit',
    'taxa_adm', 'inf_taxa_adm', 'vl_taxa_perfm', 'ds_taxa_perfm', 'trib_lprazo',
    'dt_fim'
]

In [5]:
from copy import copy
tables = copy(_tables)

In [6]:
for table_name, df in tables.items():
    rename_dict = {}
    
    ini_matches = [c for c in df.columns if 'dt_ini' in c]
    if ini_matches:
        rename_dict[ini_matches[0]] = 'dt_ini'
        
    fim_matches = [c for c in df.columns if 'dt_fim' in c]
    if fim_matches:
        rename_dict[fim_matches[0]] = 'dt_fim'
        
    df.rename(columns=rename_dict, inplace=True)
    df = df[list(set(df.columns).intersection(final_cols+list(rename_dict.keys())))]
    if table_name in ['fi_cad_fi_hist_taxa_perfm', 'fi_cad_fi_hist_taxa_adm']:
        df.loc[:, 'dt_fim'] = None
    tables[table_name] = df

In [7]:
# for key in tables.keys():
#     cols = ['cnpj_fundo', 'dt_reg', 'dt_ini', 'dt_fim']
#     print(key, set(tables[key].columns).intersection(cols))

In [8]:
from functools import reduce
import pandas as pd

df_merged = reduce(lambda left, right: pd.merge(left, right, on=["cnpj_fundo", "dt_reg", 'dt_ini', "dt_fim"], how="outer"), tables.values())


In [17]:
df_1 = tables['fi_cad_fi_hist_denom_social']
df_2 = tables['fi_cad_fi_hist_sit']
df_3 = tables['fi_cad_fi_hist_gestor']
cnpj_fundo = '58.953.805/0001-30'
cnpj_fundo = '26.218.403/0001-03'
df_1 = df_1[df_1['cnpj_fundo'] == cnpj_fundo]
df_2 = df_2[df_2['cnpj_fundo'] == cnpj_fundo]
df_3 = df_3[df_3['cnpj_fundo'] == cnpj_fundo]
df = df_1.merge(df_2, on=["cnpj_fundo", "dt_reg", 'dt_ini', "dt_fim"], how="outer")
df = df.merge(df_3, on=["cnpj_fundo", "dt_reg", 'dt_ini', "dt_fim"], how="outer")

In [18]:
chaves = ['cnpj_fundo', 'dt_reg', 'dt_ini', 'dt_fim']
df_consolidado = df.groupby(chaves, as_index=False, dropna=False).first()

In [20]:
df

,denom_social,cnpj_fundo,dt_fim,dt_ini,dt_reg,sit,pf_pj_gestor,cpf_cnpj_gestor,gestor
0,NaN,26.218.403/0001-03,2016-12-29,2016-12-26,2016-12-26,FASE PRÉ-OPERACIONAL,NaN,NaN,NaN
1,KINEA ATLAS FUNDO DE INVESTIMENTO MULTIMERCADO,26.218.403/0001-03,2024-07-16,2016-12-26,2016-12-26,NaN,NaN,NaN,NaN
2,NaN,26.218.403/0001-03,None,2016-12-26,2016-12-26,NaN,PJ,08.604.187/0001-44,KINEA INVESTIMENTOS LTDA.
3,NaN,26.218.403/0001-03,None,2016-12-29,2016-12-26,EM FUNCIONAMENTO NORMAL,NaN,NaN,NaN
4,KINEA ATLAS FUNDO DE INVESTIMENTO FINANCEIRO M...,26.218.403/0001-03,None,2024-07-16,2016-12-26,NaN,NaN,NaN,NaN


In [19]:
df_consolidado

,cnpj_fundo,dt_reg,dt_ini,dt_fim,denom_social,sit,pf_pj_gestor,cpf_cnpj_gestor,gestor
0,26.218.403/0001-03,2016-12-26,2016-12-26,2016-12-29,None,FASE PRÉ-OPERACIONAL,None,None,None
1,26.218.403/0001-03,2016-12-26,2016-12-26,2024-07-16,KINEA ATLAS FUNDO DE INVESTIMENTO MULTIMERCADO,None,None,None,None
2,26.218.403/0001-03,2016-12-26,2016-12-26,NaN,None,None,PJ,08.604.187/0001-44,KINEA INVESTIMENTOS LTDA.
3,26.218.403/0001-03,2016-12-26,2016-12-29,NaN,None,EM FUNCIONAMENTO NORMAL,None,None,None
4,26.218.403/0001-03,2016-12-26,2024-07-16,NaN,KINEA ATLAS FUNDO DE INVESTIMENTO FINANCEIRO M...,None,None,None,None


In [10]:
df

,denom_social,cnpj_fundo,dt_fim,dt_ini,dt_reg,sit,pf_pj_gestor,cpf_cnpj_gestor,gestor
0,NaN,58.953.805/0001-30,2025-01-31,2025-01-17,2025-01-17,FASE PRÉ-OPERACIONAL,NaN,NaN,NaN
1,KINEA NIX FUNDO DE INVESTIMENTO FINANCEIRO MUL...,58.953.805/0001-30,None,2025-01-17,2025-01-17,NaN,PJ,08.604.187/0001-44,KINEA INVESTIMENTOS LTDA.
2,NaN,58.953.805/0001-30,None,2025-01-31,2025-01-17,EM FUNCIONAMENTO NORMAL,NaN,NaN,NaN


In [11]:
asdasd

NameError: name 'asdasd' is not defined

In [ ]:
df[(df['denom_social'].str.upper().str.contains('NIX'))&(df['gestor'].str.upper().str.contains('KINEA'))][['denom_social', 'cnpj_fundo' ,'gestor', 'sit', 'dt_fim']]

In [ ]:
df_merged.columns

In [ ]:
df_merged[(df_merged['denom_social'].str.upper().str.contains('NIX'))&(df_merged['gestor'].str.upper().str.contains('KINEA'))][['denom_social', 'cnpj_fundo' ,'gestor', 'sit', 'dt_fim']]

In [ ]:
tables['fi_cad_fi_hist_sit'][tables['fi_cad_fi_hist_sit']['cnpj_fundo'] == '58.953.805/0001-30']

In [ ]:
df = tables.fi_cad_fi_hist_admin